In [1]:
import re
import requests
import numpy as np
import json

In [2]:
r = requests.get('http://map.amap.com/service/subway?_1469083453978&srhdata=1100_drw_beijing.json')
r_json = json.loads(r.text)
#第一层包含：地铁名称's','i',线路列表'l','o'
#线路列表
#print(r_json['l'])

#第二层线路列表包含：站列表'st',线路简称'ln','su',线路全称'kn','c',环路标记'lo','lp','f','ls','cl','la','x','li'
#线路1名称
#line_1 = r_json['l'][0]
#print(line_1['ln'])

#第三层站列表包含：'rs','udpx','su','udsu',站名称'n','sid','p','r','udsi','t','si',经纬度'sl','udli','poiid','lg',站名称拼音'sp'
#print(r_json['l'][0]['st'][0])

#线路1上的站名称
#line_1_station_name_list = ','.join(i['n'] for i in line_1['st'])
#print("线路'{}'上的站名称:{}".format(line_1['kn'],line_1_station_name_list))
#[[r_json['l'][i]['ln'],r_json['l'][i]['lo']] for i in range(len(r_json['l']))]

In [3]:
def get_lines_stations_info(text):
    # 请在这里写上你的代码
    pass
    obj = json.loads(text)   
    # 遍历text格式数据，组成地点数据结构
    # 所有线路信息的dict：key：线路名称；value：站点名称list
    lines_info = {}    
    # 所有站点信息的dict：key：站点名称；value：站点坐标(x,y)
    stations_info = {}
    #for i in range(len(lines_list)):
        # 你可能需要思考的几个问题，获取「地铁线路名称，站点信息list，站名，坐标(x,y)，数据加入站点的信息dict，将数据加入地铁线路dict」
    pass
    for line in obj['l']:
        line_name = line['kn']
        lines_info[line_name] = [i['n'] for i in line['st']]
        for i in line['st']:
            x_y = re.findall("(\d+.\d+),(\d+.\d+)",i['sl'])[0]
            stations_info[i['n']] = tuple(map(float,x_y))
    #环路
    lines_info['地铁10号线'].append("巴沟")
    lines_info['地铁2号线'].append('积水潭')
    return lines_info, stations_info

lines_info, stations_info = get_lines_stations_info(r.text)
#lines_info

In [4]:
#stations_info

In [5]:
from collections import defaultdict
# 根据线路信息，建立站点邻接表dict
def get_neighbor_info(lines_info):
    pass
    # 把str2加入str1站点的邻接表中
    def add_neighbor_dict(info, str1, str2):
        # 请在这里写代码
        pass
        info[str1].append(str2) 
    
    neighbor_info = defaultdict(list)
    for station_list in lines_info.values():
        for i in range(len(station_list)):
            station = station_list[i]
            if i == 0:
                add_neighbor_dict(neighbor_info,station,station_list[i+1])
            elif i == len(station_list)-1:
                add_neighbor_dict(neighbor_info,station,station_list[i-1])
            else:
                add_neighbor_dict(neighbor_info,station,station_list[i-1])
                add_neighbor_dict(neighbor_info,station,station_list[i+1])
    return neighbor_info
        
neighbor_info = get_neighbor_info(lines_info)
#neighbor_info

In [6]:
# 画地铁图
import networkx as nx
import matplotlib.pyplot as plt,mpld3

# 如果汉字无法显示，请参照
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['font.family']='sans-serif'
plt.figure(figsize=(30, 20))#画布大小
station_connection_graph = nx.Graph(neighbor_info)
colors = range(len(list(station_connection_graph.edges)))
#edges = nx.draw_networkx_edges(station_connection_graph,stations_info,edge_color=colors,width=2)
#plt.colorbar(edges)
nx.draw_networkx(station_connection_graph,stations_info,with_labels=True,node_size=10,font_size=8)
plt.show()

<Figure size 3000x2000 with 1 Axes>

In [7]:
def check_station_name(station):
    if station not in stations_info:
        return stations_info
    return True
#print(check_station_name('前门'))

In [8]:
#获取站点直线距离
def get_distance(stations_info,current_station,to_station):
    current_x_y = stations_info[current_station]
    to_x_y = stations_info[to_station]
    return geo_distance(current_x_y,to_x_y)
#get_distance(stations_info,'前门','中关村')

In [9]:
#获取站点所在线路
def get_lines_by_station(lines_info,station):
    lines = []
    for line in lines_info:
        if station in lines_info[line]:
            lines.append(line)
    return lines
#get_lines_by_station(lines_info,'中关村')

In [10]:
from collections import defaultdict
#获取换乘站
def get_crossLine_station(lines_info):
    station_in_lines = defaultdict(list) 
    crossLine_station = defaultdict(list) 
    for line in lines_info:
        line_station_list = lines_info[line]
        for station in lines_info[line]:
            station_in_lines[station].append(line)
    for station in station_in_lines:
        if len(station_in_lines[station])>1:
            crossLine_station[station] = station_in_lines[station]
    return crossLine_station
crossLine_station = get_crossLine_station(lines_info)
#crossLine_station

In [11]:
#获取交叉线路
def get_crossLine(lines_info):
    crossLine_station = get_crossLine_station(lines_info)
    crossLine = defaultdict(list)
    for line in lines_info:
        temp = set()
        for station in crossLine_station:
            if line in crossLine_station[station]:
                for i in range(len(crossLine_station[station])):
                    if line == crossLine_station[station][i]: continue
                    temp.add(crossLine_station[station][i])
        crossLine[line]=list(temp)
    return crossLine
#get_crossLine(lines_info)

In [12]:
def get_next_station_DFS_ALL(node, neighbor_info, to_station):
    pass
    neighbors = neighbor_info[to_station]
    smallest_index = node.index(to_station)
    for neighbor in neighbors:
        smallest_index = min(smallest_index,node.index(neighbor))
    return node[smallest_index]
#get_next_station_DFS_ALL(path_All,neighbor_info,'中关村')

In [13]:
# 你可以用递归查找所有路径
def get_path_DFS_ALL(lines_info, neighbor_info, from_station, to_station):
    # 递归算法，本质上是深度优先
    # 遍历所有路径
    # 这种情况下，站点间的坐标距离难以转化为可靠的启发函数，所以只用简单的BFS算法
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    stack_node = [from_station]
    visited = set()
    node = []
    path = []
    while stack_node:
        current = stack_node.pop(-1)
        if current not in visited:node.append(current)
        visited.add(current)
        neighbors = neighbor_info[current]
        for neighbor in neighbors:
            if neighbor in visited:continue
            stack_node.append(neighbor)
    #从终点站逆推
    recursion_station = to_station
    while recursion_station != from_station:
        path=[recursion_station]+path
        recursion_station = get_next_station_DFS_ALL(node,neighbor_info,recursion_station)
    return [from_station]+path
print('DFS_ALL:')
get_path_DFS_ALL(lines_info,neighbor_info,'前门','中关村')

DFS_ALL:


['前门',
 '和平门',
 '宣武门',
 '西单',
 '灵境胡同',
 '西四',
 '平安里',
 '车公庄',
 '车公庄西',
 '白石桥南',
 '国家图书馆',
 '魏公村',
 '人民大学',
 '海淀黄庄',
 '中关村']

In [14]:
#不遍历，遇到终点马上输出路线
def get_path_DFS(neighbor_info,from_station,to_station):
    stack_node = [from_station]
    visited = set()
    path = []
    while stack:
        current = stack.pop(-1)
        visited.add(current)
        path.append(current)
        neighbors = neighbor_info[current]
        for neighbor in neighbors:
            if neighbor in visited:continue
            if neighbor == to_station:
                path.append(to_station)
                return path
            stack = stack+[neighbor]

In [15]:
#get_path_DFS(neighbor_info,'石门','燕山')

In [16]:
#  你也可以使用第二种算法：没有启发函数的简单宽度优先

def get_path_BFS(lines_info, neighbor_info,stations_info, from_station, to_station):
    # 搜索策略：以站点数量为cost（因为车票价格是按站算的）
    # 这种情况下，站点间的坐标距离难以转化为可靠的启发函数，所以只用简单的BFS算法
    # 由于每深一层就是cost加1，所以每层的cost都相同，算和不算没区别，所以省略
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    queue_path = [[from_station]]
    #visited是关键，若某一探索线路的下一站已经在visited中，
    #说明之前的探索线路已经到达，而且层数比它还小，因此舍弃这条线路，迅速减少stack_path规模
    visited = set()
    while queue_path:
        path = queue_path.pop(0)
        node = path[-1]
        neighbors = neighbor_info[node]
        for neighbor in neighbors:
            if neighbor in path or neighbor in visited:continue
            if neighbor==to_station:
                return path+[neighbor]
            queue_path.append(path+[neighbor])
            visited.add(neighbor)
print('BFS:')
get_path_BFS(lines_info,neighbor_info,stations_info,'前门','中关村')

BFS:


['前门',
 '和平门',
 '宣武门',
 '长椿街',
 '复兴门',
 '阜成门',
 '车公庄',
 '西直门',
 '大钟寺',
 '知春路',
 '知春里',
 '海淀黄庄',
 '中关村']

In [17]:
#任选某一起点，寻找最深层数的站点及其路径
def get_deapest_station(neighbor_info, from_station):
    pass
    stack_path = [[from_station]]
    path = []
    #visited是关键，若某一探索线路的下一站已经在visited中，
    #说明之前的探索线路已经到达，而且层数比它还小，因此舍弃这条线路，迅速减少stack_path规模
    visited = set()
    while stack_path:
        path = stack_path.pop(0)
        node = path[-1]
        neighbors = neighbor_info[node]
        for neighbor in neighbors:
            if neighbor in path or neighbor in visited:continue
            stack_path.append(path+[neighbor])
            visited.add(neighbor)
    return path
#get_path_BFS_ALL(neighbor_info,'八宝山')

In [18]:
import math

def geo_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [19]:
def get_next_station(came_from,to_station):
    return came_from[to_station]
def priority_Astar(stations_info,neighbor,to_station):
    return get_distance(stations_info,neighbor,to_station)

In [20]:
# 你还可以用第三种算法：以路径路程为cost的启发式搜索
from queue import PriorityQueue
import pandas as pd
def get_path_Astar(lines_info, neighbor_info, stations_info, from_station, to_station):
    # 搜索策略：以路径的站点间直线距离累加为cost，以当前站点到目标的直线距离为启发函数
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    #记录遍历站点
    frontier = PriorityQueue()
    frontier.put((0,from_station))
    #记录站点（如A）对应上一站的名称。如果current站点的下一站为A，且累计cost比A当前的上一站cost少，更新current站点为站A的上一站。
    came_from = {}
    came_from[from_station]=from_station
    #记录站点（如A）累计cost。
    cost = {}
    cost[from_station]=0
    while frontier:
        current_station = frontier.get()[1]
        if current_station == to_station:
            pre_station = to_station
            full_path = [to_station]
            while pre_station!=from_station:
                pre_station = get_next_station(came_from,pre_station)
                full_path = [pre_station]+full_path
            return full_path
        neighbors = neighbor_info[current_station]
        for neighbor in neighbors:
            #从cost[current_station]获取当前站点累计cost，再加上当前站点到邻接点的距离，获得邻接点的新累计cost。
            new_cost = get_distance(stations_info,current_station,neighbor)+cost[current_station]
            #如果cost中没有该邻接点，说明第一次到达。如果有，比较此次新cost与之前的累计cost的大小，小的话，替换邻接点的上一站为current站，并更新cost。
            if neighbor not in cost or cost[neighbor]>new_cost:
                cost[neighbor] = new_cost
                came_from[neighbor] = current_station
                priority = priority_Astar(stations_info,neighbor,to_station)
                frontier.put((priority,neighbor))
print('Astar:')
get_path_Astar(lines_info,neighbor_info,stations_info,'前门','中关村')

Astar:


['前门',
 '和平门',
 '宣武门',
 '长椿街',
 '复兴门',
 '阜成门',
 '车公庄',
 '车公庄西',
 '白石桥南',
 '国家图书馆',
 '魏公村',
 '人民大学',
 '海淀黄庄',
 '中关村']

In [21]:
def get_path_Shortest_Path(lines_info,neighbor_info, stations_info, from_station, to_station):
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    #记录遍历站点
    frontier = PriorityQueue()
    frontier.put((0,from_station))
    #记录站点（如A）对应上一站的名称。如果current站点的下一站为A，且累计cost比A当前的上一站cost少，更新current站点为站A的上一站。
    came_from = {}
    came_from[from_station]=from_station
    #记录站点（如A）累计cost。
    cost = {}
    cost[from_station]=0
    while frontier:
        current_station = frontier.get()[1]
        if current_station == to_station:
            pre_station = to_station
            full_path = [to_station]
            while pre_station!=from_station:
                pre_station = get_next_station(came_from,pre_station)
                full_path = [pre_station]+full_path
            return full_path
        neighbors = neighbor_info[current_station]
        for neighbor in neighbors:
            #从cost[current_station]获取当前站点累计cost，再加上当前站点到邻接点的距离，获得邻接点的新累计cost。
            new_cost = get_distance(stations_info,current_station,neighbor)+cost[current_station]
            #根据不同策略调整priority值
            if neighbor not in cost or cost[neighbor]>new_cost:
                cost[neighbor] = new_cost
                came_from[neighbor] = current_station
                priority = get_distance(stations_info,current_station,neighbor)
                frontier.put((priority,neighbor))
print('Shortest_Path:')
get_path_Shortest_Path(lines_info,neighbor_info,stations_info,'前门','中关村')

Shortest_Path:


['前门',
 '和平门',
 '宣武门',
 '西单',
 '灵境胡同',
 '西四',
 '平安里',
 '新街口',
 '西直门',
 '大钟寺',
 '知春路',
 '知春里',
 '海淀黄庄',
 '中关村']

In [22]:
def is_exchange_station(lines_info,pre_station,current_station,neighbor):
    lines_pre_station = get_lines_by_station(lines_info,pre_station)
    lines_current_station = get_lines_by_station(lines_info,current_station)
    lines_neighbor = get_lines_by_station(lines_info,neighbor)
    line_from = set(lines_pre_station)&set(lines_current_station)
    line_to=set(lines_current_station)&set(lines_neighbor)
    return 0 if line_from == line_to else 1

In [23]:
def get_path_Minimum_Transfer(lines_info,neighbor_info, stations_info, from_station, to_station):
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    #记录遍历站点
    frontier = PriorityQueue()
    frontier.put((0,from_station))
    #记录站点（如A）对应上一站的名称。如果current站点的下一站为A，且累计cost比A当前的上一站cost少，更新current站点为站A的上一站。
    came_from = {}
    came_from[from_station]=from_station
    #记录站点（如A）累计cost。
    cost = {}
    cost[from_station]=0
    while frontier:
        current_station = frontier.get()[1]
        if current_station == to_station:
            pre_station = to_station
            full_path = [to_station]
            while pre_station!=from_station:
                pre_station = get_next_station(came_from,pre_station)
                full_path = [pre_station]+full_path
            return full_path
        neighbors = neighbor_info[current_station]
        for neighbor in neighbors:
            #从cost[current_station]获取当前站点累计cost，再加上当前站点到邻接点的距离，获得邻接点的新累计cost。
            new_cost = is_exchange_station(lines_info,came_from[current_station],current_station,neighbor)+cost[current_station]
            #根据不同策略调整priority值
            if neighbor not in cost or cost[neighbor]>new_cost:
                cost[neighbor] = new_cost
                came_from[neighbor] = current_station
                priority = is_exchange_station(lines_info,came_from[current_station],current_station,neighbor)
                frontier.put((priority,neighbor))
print('Minimum_Transfer_Path:')
get_path_Minimum_Transfer(lines_info,neighbor_info,stations_info,'前门','中关村')

Minimum_Transfer_Path:


['前门',
 '和平门',
 '宣武门',
 '长椿街',
 '复兴门',
 '阜成门',
 '车公庄',
 '西直门',
 '动物园',
 '国家图书馆',
 '魏公村',
 '人民大学',
 '海淀黄庄',
 '中关村']

In [88]:
#Astar和Shortest都运用了贪心算法和Dijkstra算法思想，但缺点是只能应用于每一站的cost值是固定的，若下一站的cost与前一站的路径有关，则不能应用。
def get_path_Comprehensive1(lines_info,neighbor_info, stations_info, from_station, to_station):
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    #记录遍历站点
    frontier = PriorityQueue()
    frontier.put((0,[from_station]))
    while frontier:
        current = frontier.get()
        cost = current[0]
        current_station = current[1][-1]
        if current_station == to_station:
            return current[1]
        neighbors = neighbor_info[current_station]
        #从cost[current_station]获取当前站点累计cost，再加上当前站点到邻接点的距离，获得邻接点的新累计cost。
        for neighbor in neighbors:
            pathes = current[1].copy()
            if neighbor in pathes:continue
            distance_cost = get_distance(stations_info,current_station,neighbor)
            exchange_cost = 0
            if len(pathes) > 1:
                exchange_cost = is_exchange_station(lines_info,pathes[-2],current_station,neighbor)
            new_cost = distance_cost+cost+exchange_cost
            pathes.append(neighbor)
            frontier.put((new_cost,pathes))

In [121]:
print('Comprehensive_Path:')
get_path_Comprehensive(lines_info,neighbor_info,stations_info,'奥体中心', '天安门东')

Comprehensive_Path:


['奥体中心',
 '北土城',
 '安华桥',
 '安德里北街',
 '鼓楼大街',
 '什刹海',
 '南锣鼓巷',
 '东四',
 '灯市口',
 '东单',
 '王府井',
 '天安门东']

In [ ]:
def get_path_Comprehensive2(lines_info,neighbor_info, stations_info, from_station, to_station):
    # 检查输入站点名称
    pass
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    #记录遍历站点
    frontier = PriorityQueue()
    frontier.put((0,[from_station]))
    node_cost = {}
    node_cost[from_station] = [[0,from_station,from_station,from_station]]
    while frontier:
        current = frontier.get()
        cost = current[0]
        current_station = current[1][-1]
        if current_station == to_station:
            return current[1]
        neighbors = neighbor_info[current_station]
        #从cost[current_station]获取当前站点累计cost，再加上当前站点到邻接点的距离，获得邻接点的新累计cost。
        for neighbor in neighbors:
            pathes = current[1].copy()
            if neighbor in pathes:continue
            distance_cost = get_distance(stations_info,current_station,neighbor)
            exchange_cost = 0
            if len(pathes) > 1:
                exchange_cost = is_exchange_station(lines_info,pathes[-2],current_station,neighbor)
            new_cost = distance_cost+cost+exchange_cost
            pathes.append(neighbor)
            frontier.put((new_cost,pathes))

In [112]:
def path_distance(path):
    value = 0
    for i in range(len(path)-1):
        value+=get_distance(stations_info,path[i],path[i+1])
    return value
def path_transfer_num(path):
    value = 0
    path.append(path[-1])
    for i in range(len(path)-2):
        value+=is_exchange_station(lines_info,path[i],path[i+1],path[i+2])
    path.pop(-1)
    return value
def path_station_num(path):
    return len(path)
def collection_dict(path,strategy_name,show_path=False):
    value = {}
    value['strategy_name'] = strategy_name
    value['distance'] = path_distance(path)
    value['transfer_num'] = path_transfer_num(path)
    value['station_num'] = path_station_num(path)
    if show_path:
        value['path'] = path
    return value

In [115]:
def get_path_collection(lines_info,neighbor_info, stations_info, from_station, to_station):
    if check_station_name(from_station)==None or check_station_name(to_station)==None:
        return
    path_BFS = get_path_BFS(lines_info,neighbor_info, stations_info, from_station, to_station)
    path_Astar = get_path_Astar(lines_info,neighbor_info, stations_info, from_station, to_station)
    path_Shortest_Path = get_path_Shortest_Path(lines_info,neighbor_info, stations_info, from_station, to_station)
    path_Minimum_Transfer = get_path_Minimum_Transfer(lines_info,neighbor_info, stations_info, from_station, to_station)
    path_Comprehensive = get_path_Comprehensive(lines_info,neighbor_info,stations_info,from_station,to_station)
    dict_BFS = collection_dict(path_BFS,'BFS')
    dict_Astar = collection_dict(path_Astar,'Astar')
    dict_Shortest_Path = collection_dict(path_Shortest_Path,'Shortest_Path')
    dict_Minimum_Transfer = collection_dict(path_Minimum_Transfer,'Minimum_Transfer')
    dict_Comprehensive = collection_dict(path_Comprehensive,'Comprehensive')
    return dict_BFS,dict_Astar,dict_Minimum_Transfer,dict_Shortest_Path,dict_Comprehensive

In [118]:
get_path_collection(lines_info,neighbor_info,stations_info,'前门','中关村')

({'strategy_name': 'BFS',
  'distance': 11.745804783103985,
  'transfer_num': 3,
  'station_num': 13},
 {'strategy_name': 'Astar',
  'distance': 11.918164107459237,
  'transfer_num': 3,
  'station_num': 14},
 {'strategy_name': 'Minimum_Transfer',
  'distance': 12.028634537274339,
  'transfer_num': 1,
  'station_num': 14},
 {'strategy_name': 'Shortest_Path',
  'distance': 11.73472648997079,
  'transfer_num': 4,
  'station_num': 14},
 {'strategy_name': 'Comprehensive',
  'distance': 12.017556244141144,
  'transfer_num': 1,
  'station_num': 15})